In [2]:
import numpy as np
from sympy import var, Lambda, exp, log, sin, cos, tan, sqrt, ln
from math import inf
from fractions import Fraction
import matplotlib.pyplot as plt

# Resolvendo exercicios da aula

# Fluxo de carga

In [14]:
y12 = -1/(0.01+0.05j)
y21 = -1/(0.01+0.05j)

y11 = -y12
y22 = -y12
n = 2 #ordem da matriz

#Matriz de admitância
Y = np.empty((n), dtype=complex)
Y = np.array([[y11, y12],
              [y21, y22]])


print('Y = ', Y)

Y =  [[ 3.84615385-19.23076923j -3.84615385+19.23076923j]
 [-3.84615385+19.23076923j  3.84615385-19.23076923j]]


In [3]:
x,y = var('teta2 V2')

Pk = Lambda((teta2, V2), )
#x = [[tetan], [Vn]] em rad e pu
x0 = np.zeros((2,1))
x0[0,0] = 0
x0[1,0] = 1

#x = [[DeltaP], [DeltaQ]] em pu
y0 = np.zeros((2,1))
y0[0,0] = 0
y0[1,0] = 0


[[0.]
 [1.]]


In [7]:
y12 = complex(0.012, 0.3)**(-1)
y23 = complex(0.011, 0.8)**(-1)

Y = [[y12, -y12, 0],
     [-y12, y12 + y23, -y23],
     [0, -y23, y23]]